# Projeto 1 - Ciência dos Dados

Nome: Luca Coutinho Melão

Nome: Thiago de Donato Youssef

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [11]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [15]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [16]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,a sony anunciando o preço do ps5 hoje\nhttps:/...,0
1,vou prar as minhas atividades de desenvolvimen...,0
2,alguma live do evento do ps5 com tradução em p...,1
3,"@thecure70s pô pior que n sei, mas vai ser dep...",1
4,@fiaspo olha... e se rolar gameplay de tlou 2 ...,1


In [17]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,daqui a meia hora tem novidades do #ps5! ansio...,1
1,"@smashrs to plugado, ansioso pela decepção em ...",1
2,"eu: n vou ver o ps5 showcase, tenho muita cois...",0
3,será que vai rolar o preço do ps5? mesmo saben...,1
4,rt @ojovemgamer: quando a sony divulgar o preç...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O PS5 É O NOVO CONSOLE DE JOGOS DA SONY, COM SEU LANÇAMENTO AO FINAL DE 2020.

RELEVANTE

Tweets que expressam o sentimento do usuário com relação ao console, remetem geralmente às expectativas sobre os preços, jogos mais aguardados, comparações com a concorrência.

IRRELEVANTE

Tweets que não estão relacionados ao produto, ou que não agregam nada à análise de sentimentos sobre ele. Tais como: anúncios de lives e comentários sem significado.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**